In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


datadir = "../data/"

In [9]:
ds = xr.open_mfdataset(datadir + "originaldata/" + "etopo*.nc")
ds

<xarray.Dataset>
Dimensions:  (lat: 900, lon: 10800)
Coordinates:
  * lat      (lat) float32 60.0 60.03 60.07 60.1 ... 89.87 89.9 89.93 89.97
  * lon      (lon) float32 -180.0 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Data variables:
    topo     (lat, lon) float32 dask.array<chunksize=(900, 10800), meta=np.ndarray>
Attributes:
    Conventions:         CF-1.0
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  60.0
    geospatial_lat_max:  89.96666717529297
    geospatial_lon_min:  -180.0
    geospatial_lon_max:  179.96665954589844

In [11]:
ds = ds.sel(lat=slice(60, 90))
ds

<xarray.Dataset>
Dimensions:  (lat: 900, lon: 10800)
Coordinates:
  * lat      (lat) float32 60.0 60.03 60.07 60.1 ... 89.87 89.9 89.93 89.97
  * lon      (lon) float32 -180.0 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Data variables:
    topo     (lat, lon) float32 dask.array<chunksize=(900, 10800), meta=np.ndarray>
Attributes:
    Conventions:         CF-1.0
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  60.0
    geospatial_lat_max:  89.96666717529297
    geospatial_lon_min:  -180.0
    geospatial_lon_max:  179.96665954589844

In [12]:
ds = -ds.where(ds.topo < 0)
ds = ds.rename({"topo" : "depth"})
ds

<xarray.Dataset>
Dimensions:  (lat: 900, lon: 10800)
Coordinates:
  * lat      (lat) float32 60.0 60.03 60.07 60.1 ... 89.87 89.9 89.93 89.97
  * lon      (lon) float32 -180.0 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Data variables:
    depth    (lat, lon) float32 dask.array<chunksize=(900, 10800), meta=np.ndarray>
Attributes:
    Conventions:         CF-1.0
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  60.0
    geospatial_lat_max:  89.96666717529297
    geospatial_lon_min:  -180.0
    geospatial_lon_max:  179.96665954589844

In [ ]:
fig, ax = plt.subplots(1, 1,
                       figsize=(18, 14),
                       subplot_kw={"projection" : ccrs.NorthPolarStereo()}
                      )
ax.coastlines()
ax.gridlines()
ax.set_extent([-180, 180, 60, 90], crs=ccrs.PlateCarree())


cmap = plt.get_cmap('Blues', 50)

c = ax.pcolormesh(ds.lon, ds.lat, ds.depth.isel(lon=slice(0, -1), lat=slice(0, -1)), 
                  shading='flat',
                  transform=ccrs.PlateCarree(),
                  vmin=0, 
                  vmax=5600,
                  cmap=cmap,
                  rasterized=True
           )
ax.set_title("Depth")
fig.colorbar(c,
             shrink=0.5,
             label=r"$\nabla$ sh [m]",
             extend="both"
            )

In [13]:
ds.to_netcdf(datadir + "bathymetry.nc")